In [1]:
!pip install imagehash

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.9/60.9 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.5/296.5 kB 38.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.3/16.3 MB 115.5 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: numpy
    Found existing installation: numpy 1.22.4
    Uninstalling numpy-1.22.4:
      Successfully uninstalled numpy-1.22.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
mkl-fft 1.3.10 requires mkl, which is not installed.
hdijupyterutils 0.21.0 requires pandas<2.0.0,>=0.17.1, but you have pandas 2.2.2 which is incompatible.
numba 0.60.0 requires numpy<2.1,>=1.22, but you have numpy 2.1.1 which is incompatible.
sagemaker 2.229.0 requires numpy<2.0,>=1.9.0, but you have numpy 2.1.1 which is incompatible.
sparkmagic 0.21.0 requires pandas<2.0.0,>=0.17.1, but you have pandas 2.2.2 which is

In [8]:
import pandas as pd
import imagehash
from PIL import Image
import aiohttp
import asyncio
from io import BytesIO
from tqdm.notebook import tqdm
import boto3
import gc
import json
from botocore.exceptions import ClientError

dynamodb = boto3.resource('dynamodb', region_name='us-east-1')
table = dynamodb.Table('hash_store')

df = pd.read_csv('image_data_b1.csv')
df = df.dropna(subset=['small_image'])

base_url = "https://d1it09c4puycyh.cloudfront.net"
dimensions = "707x1000"

async def fetch_image(session, url):
    async with session.get(url) as response:
        if response.status == 200:
            img_data = await response.read()
            return Image.open(BytesIO(img_data)).convert('RGB')
        else:
            raise IOError(f"Failed to fetch image from {url}, status code: {response.status}")

def calculate_phash(img):
    long_side = max(img.size)
    ratio = 512 / long_side
    new_size = (int(img.size[0] * ratio), int(img.size[1] * ratio))
    img = img.resize(new_size, Image.Resampling.LANCZOS)
    
    new_img = Image.new('RGB', (512, 512), (255, 255, 255))
    paste_pos = ((512 - new_size[0]) // 2, (512 - new_size[1]) // 2)
    new_img.paste(img, paste_pos)
    
    return imagehash.phash(new_img)

async def process_batch(batch_df, batch_number):
    async with aiohttp.ClientSession() as session:
        failed_rows = []
        for index, row in tqdm(batch_df.iterrows(), total=len(batch_df), desc=f"Processing Batch {batch_number}"):
            image_url = f"{base_url}/{dimensions}/catalog/product{row['small_image'].strip()}"
            try:
                img = await fetch_image(session, image_url)
                phash = calculate_phash(img)
                
                item = {
                    'entity_id': row['entity_id'],
                    'sku': row['sku'],
                    'small_image': row['small_image'],
                    'phash': str(phash)
                }
                try:
                    table.put_item(Item=item)  # Insert into DynamoDB
                except ClientError as e:
                    print(f"Failed to insert item into DynamoDB: {e}")
                    failed_rows.append({'entity_id': row['entity_id']})

            except Exception as e:
                print(f"Failed to process image for SKU {row['sku']}: {e}")
                failed_rows.append({'entity_id': row['entity_id']})

        print(f"Batch {batch_number} processing complete.")
        if failed_rows:
            print(f"Failed to process {len(failed_rows)} items.")
        
        gc.collect()  # Freeing up memory

async def process_all_batches(start_batch=1, end_batch=None):
    batch_size = 100
    num_batches = len(df) // batch_size + (1 if len(df) % batch_size != 0 else 0)
    
    if end_batch is None:
        end_batch = num_batches

    for i in range(start_batch - 1, end_batch):
        batch_df = df.iloc[i * batch_size:(i + 1) * batch_size]
        await process_batch(batch_df, i + 1)
        gc.collect()

In [9]:
await process_all_batches(start_batch=1)

Processing Batch 1:   0%|          | 0/100 [00:00<?, ?it/s]

Batch 1 processing complete.


Processing Batch 2:   0%|          | 0/100 [00:00<?, ?it/s]

Batch 2 processing complete.


Processing Batch 3:   0%|          | 0/100 [00:00<?, ?it/s]

...

Batch 4 processing complete.


Processing Batch 5:   0%|          | 0/100 [00:00<?, ?it/s]

Batch 5 processing complete.


Processing Batch 6:   0%|          | 0/100 [00:00<?, ?it/s]

CancelledError: 